### Get Developable Area by Grid
This notebook is the first notebook in the resourceAssessment workflow. In this notebook we take two file geodatabases (1) **resourceAssessment.gdb**, contains 4 polygon feature classes representing suitable areas for industrial wind and solar development in the CONUS according to two different models (WU, ZRM) (2) **grids.gdb** contains fishnet polygons derived from the geometries of two different atmospheric analysis datasets (cesm2, era5). For each developable area dataset, for each grid geometry, we calculate the total developable area per grid cell, and the percent developable area per grid cell. Results are saved to **resourceAssessementResults.gdb**. 

In [26]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define useful functions

In [27]:
def intersect(input_fc, grid_fc, output_fc): # input fc should the result of a dissolve operation
    
    # Intersect the dissolved feature class with the grid feature class
    intersect_fc = arcpy.analysis.Intersect([input_fc, grid_fc], arcpy.Geometry(), "ALL", None, "INPUT")

    # Copy the intersected feature class to the output location
    arcpy.management.CopyFeatures(intersect_fc, output_fc)

    print(f"Intersect completed for {input_fc} and {grid_fc}")

In [28]:
def calculate_shape_field(input_fc):

    # create new field with double data type
    arcpy.management.AddField(input_fc, "CPA_Area", "DOUBLE")
    
    # write expression for the calculation
    expression = "float(!SHAPE.AREA!)"
    # Calculate the shape field using the SHAPE@AREA token
    arcpy.management.CalculateField(input_fc, "CPA_Area", expression, "PYTHON")

    print(f"Shape field calculated for {input_fc}!")

In [29]:
def convert_to_point_feature_class(input_fc, output_fc):
    # Convert the input feature class to a point feature class
    arcpy.management.FeatureToPoint(input_fc, output_fc, "INSIDE")

    print(f"Converted to point feature class: {output_fc}")

In [30]:
def perform_join(point_fc, grid_fc, output_fc):
    # Join the point feature class with the grid feature class
    arcpy.analysis.SpatialJoin(grid_fc, point_fc, output_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL")

    print(f"Join completed for {point_fc} and {grid_fc}")

In [31]:
def calculate_area_pct(joined_fc):
    # Add a new field to store the area percentage
    arcpy.management.AddField(joined_fc, "CPA_pct", "DOUBLE")

    # Calculate the area percentage
    expression = "None if (!{0}! is None or !{1}! is None or !{1}! == 0) else round(!{0}! / !{1}!, 2)".format("CPA_Area", "Shape_Area")
    arcpy.management.CalculateField(joined_fc, "CPA_pct", expression, "PYTHON")

    print(f"Area percentage calculated for {joined_fc}!")

In [32]:
# grab all feature classes from a folder or geodatabase
def get_feature_classes(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace

        # Get a list of all feature classes in the workspace
        feature_classes = arcpy.ListFeatureClasses()
        feature_classes = [os.path.join(arcpy.env.workspace, r) for r in feature_classes]

        # Return the list of feature classes
        return feature_classes

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [33]:
def repair_geometry(input_fc, output_fc=None):
    # Check if the input feature class exists
    if not arcpy.Exists(input_fc):
        raise ValueError("The input feature class does not exist.")

    # Set the output feature class path if not provided
    if output_fc is None:
        output_fc = input_fc

    # Ensure overwrite output is enabled
    arcpy.env.overwriteOutput = True

    try:
        # Repair the geometry using arcpy.RepairGeometry_management
        arcpy.RepairGeometry_management(input_fc)
        arcpy.AddMessage("Geometry repaired successfully.")

    except arcpy.ExecuteError:
        # If there's an error, print the error message
        error_message = arcpy.GetMessages(2)
        arcpy.AddError(error_message)
        raise Exception("Geometry repair failed.")

    return output_fc

#### Set input and output paths

In [34]:
# Set the workspace for the geodatabase containing the input feature classes
arcpy.env.workspace = r"C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\data\\resourceAssessment.gdb"

In [35]:
# Set the workspace for the geodatabase containing the grid feature classes
grid_workspace = r"C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\data\\grids.gdb"

In [36]:
# Set the output workspace for the resulting feature classes
output_workspace = r"C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\scratch.gdb"

In [37]:
# set the workspace for the final output
final_workspace = r"C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\\data\\resourceAssessmentResults.gdb"

#### Get total Inclusion Area/Candidate Project Area by grid cell

In [38]:
# Get a list of feature classes in the input geodatabase
input_feature_classes = arcpy.ListFeatureClasses()
input_feature_classes = [os.path.join(arcpy.env.workspace,r) for r in input_feature_classes]
input_feature_classes

['C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\resourceAssessment.gdb\\CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved',
 'C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\resourceAssessment.gdb\\CandidateProjectArea_Wind_SL1_WU_projected_clipped_dissolved',
 'C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\resourceAssessment.gdb\\InclusionArea_Wind_SL1_ZRM_filters_dissolved',
 'C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\resourceAssessment.gdb\\InclusionArea_Solar_SL1_ZRM_filters_dissolved']

In [39]:
 # Get a list of feature classes in the grid geodatabase
grid_feature_classes = get_feature_classes(grid_workspace)
grid_feature_classes

['C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\grids.gdb\\era5_base_grid',
 'C:\\\\Users\\Zachary\\ASSET\\\\resourceAssessment\\\\analysis\\data\\\\grids.gdb\\cesm2_base_grid']

In [40]:
blah = os.path.splitext(os.path.basename(grid_feature_classes[0]))[0]
blah

'era5_base_grid'

In [41]:
# Iterate over the input feature classes
for input_fc in input_feature_classes:
    
    # Get the base name of the input feature class
    input_name = os.path.splitext(os.path.basename(input_fc))[0]

    # Iterate over the grid feature classes
    for grid_fc in grid_feature_classes:

        # Get the base name of the grid feature class
        grid_name = os.path.splitext(os.path.basename(grid_fc))[0]
        
        # Construct the full path to the output feature class in the output workspace
        output_fc = os.path.join(output_workspace, f"{input_name}_intersect_{grid_name}")

        # Apply intersect function
        intersect(input_fc, grid_fc, output_fc)

        # Apply calculate shape field function
        calculate_shape_field(output_fc)

        # Convert the output feature class to a point feature class
        point_name = os.path.splitext(output_fc)[0] + '_point'
        point_fc = os.path.join(output_workspace, point_name)
        convert_to_point_feature_class(output_fc, point_fc)

        # Perform the join between the point feature class and the grid feature class
        joined_name = os.path.splitext(point_fc)[0] + '_joined'
        joined_fc = os.path.join(final_workspace, joined_name)
        perform_join(point_fc, grid_fc, joined_fc)

        # Calculate the area percentage in the joined feature class
        calculate_area_pct(joined_fc)

Intersect completed for C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\data\\resourceAssessment.gdb\CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved and C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\data\\grids.gdb\era5_base_grid
Shape field calculated for C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\scratch.gdb\CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved_intersect_era5_base_grid!
Converted to point feature class: C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\scratch.gdb\CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved_intersect_era5_base_grid_point
Join completed for C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\scratch.gdb\CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved_intersect_era5_base_grid_point and C:\\Users\Zachary\ASSET\\resourceAssessment\\analysis\data\\grids.gdb\era5_base_grid
Area percentage calculated for C:\\Users\\Zachary\\ASSET\\resourceAssessme

### Rename Area and Pct fields

In [42]:
arcpy.env.workspace = output_workspace
# Specify the keyword to filter the feature classes
wildCard = "*joined*"

# Specify the field name to be renamed
field_to_rename_1 = "CPA_Area"
field_to_rename_2 = "CPA_pct"

# Specify the new name for the field
new_field_name_1 = "IA_Area"
new_field_name_2 = "IA_pct"

In [43]:
# Get a list of all feature classes in workspace that contain wildcard
feature_classes = arcpy.ListFeatureClasses(wild_card=wildCard)
feature_classes

['CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved_intersect_era5_base_grid_point_joined',
 'CandidateProjectArea_Solar_SL1_WU_projected_clipped_dissolved_intersect_cesm2_base_grid_point_joined',
 'CandidateProjectArea_Wind_SL1_WU_projected_clipped_dissolved_intersect_era5_base_grid_point_joined',
 'CandidateProjectArea_Wind_SL1_WU_projected_clipped_dissolved_intersect_cesm2_base_grid_point_joined',
 'InclusionArea_Wind_SL1_ZRM_filters_dissolved_intersect_era5_base_grid_point_joined',
 'InclusionArea_Wind_SL1_ZRM_filters_dissolved_intersect_cesm2_base_grid_point_joined',
 'InclusionArea_Solar_SL1_ZRM_filters_dissolved_intersect_era5_base_grid_point_joined',
 'InclusionArea_Solar_SL1_ZRM_filters_dissolved_intersect_cesm2_base_grid_point_joined']

In [44]:
# Specify the second keyword
keyword = "Inclusion"

In [45]:
# Iterate through the feature classes
for feature_class in feature_classes:

    # Extract the feature class name
    fc_name = arcpy.Describe(feature_class).name

    # Check if keyword is present in the feature class name
    if keyword in fc_name:
        # Get the full path of the feature class
        feature_class_path = arcpy.env.workspace + "/" + feature_class
        
        # Get the list of field names in the feature class
        field_names = [field.name for field in arcpy.ListFields(feature_class_path)]

        print(field_names)
       # Check if the field exists in the feature class
        if field_to_rename_1 in field_names:
            # Rename the field
            arcpy.AlterField_management(feature_class_path, field_to_rename_1, new_field_name_1, new_field_name_1)
        else:
            print(f"Field '{field_to_rename_1}' does not exist in {feature_class}.")

        # Check if the field exists in the feature class #2
        if field_to_rename_2 in field_names:
            # Rename the field #2
            arcpy.AlterField_management(feature_class_path, field_to_rename_2, new_field_name_2, new_field_name_2)
        else:
            print(f"Field '{field_to_rename_2}' does not exist in {feature_class}.")

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'FID_1', 'CPA_Area', 'ORIG_FID', 'Shape_Length', 'Shape_Area', 'CPA_pct']
['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'FID_1', 'CPA_Area', 'ORIG_FID', 'Shape_Length', 'Shape_Area', 'CPA_pct']
['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'FID_1', 'CPA_Area', 'ORIG_FID', 'Shape_Length', 'Shape_Area', 'CPA_pct']
['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'FID_1', 'CPA_Area', 'ORIG_FID', 'Shape_Length', 'Shape_Area', 'CPA_pct']
